<div class="alert alert-block alert-success">
<h4><center>Harshil Patel (pmb5br@virginia.edu) DS 5001 Spring 2023</center></h4>
</div>

<hr>

<div class="alert alert-block alert-info">
<h3><center> The Sun Also Rises, by Ernest Hemingway</center></h3>
</div>

<hr>

<div class="alert alert-block alert-warning">
<h4><center>Notebook 6: Content</center></h4> 
<ul>
  <li>Prepare data for PCA</li>
  <li>Compute reduced version of TFIDF table</li>
  <li>PCA Process and function</li>
  <li>Visualization of PCs and Loadings</li>
  <li>Word Embedding using GENSIM and TSNE plot function</li>
  <li>Scatterplot for visualization</li>
  <li>Inspection of analogies in the generated model</li>
</ul>
    
Tables saved - corpus, TFIDF, DFIDF, BOW, LIB, tokens_final
    
</div>

<hr>

### Prepare data for PCA

In [1]:
OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']

In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from scipy.linalg import norm

In [3]:
import plotly_express as px
import seaborn as sns

In [4]:
data = {'genre_id': ['fiction'],
        'author_id': ['ernest']}
LIB = pd.DataFrame(data, index=['thesunalsorises'])
LIB.index.name = 'book_id'

In [5]:
LIB

,genre_id,author_id
book_id,,
thesunalsorises,fiction,ernest


In [6]:
TOKENS = pd.read_csv(f"../data/output/nltk_tokens.csv", index_col=0)

In [7]:
TOKENS

,para_num,sent_num,token_num,pos_tuple,pos,token_str,term_str
chap_num,,,,,,,
1,0,0,0,"('Robert', 'NNP')",NNP,Robert,robert
1,0,0,1,"('Cohn', 'NNP')",NNP,Cohn,cohn
1,0,0,2,"('was', 'VBD')",VBD,was,was
1,0,0,3,"('once', 'RB')",RB,once,once
1,0,0,4,"('middleweight', 'JJ')",JJ,middleweight,middleweight
...,...,...,...,...,...,...,...
19,264,1,8,"('so', 'RB')",RB,so,so
19,264,1,9,"('?', '.')",.,?,?
19,264,1,10,"('”', 'NN')",NN,”,”


In [8]:
TOKENS = TOKENS.drop(['token_str', 'pos_tuple'], axis=1)
TOKENS

,para_num,sent_num,token_num,pos,term_str
chap_num,,,,,
1,0,0,0,NNP,robert
1,0,0,1,NNP,cohn
1,0,0,2,VBD,was
1,0,0,3,RB,once
1,0,0,4,JJ,middleweight
...,...,...,...,...,...
19,264,1,8,RB,so
19,264,1,9,.,?
19,264,1,10,NN,”


In [9]:
TOKENS = TOKENS.reset_index()
book_id = 'thesunalsorises'
TOKENS['book_id'] = book_id
TOKENS = TOKENS.set_index(['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num'])

In [10]:
TOKENS.head()

pos      term_str
book_id         chap_num para_num sent_num token_num                   
thesunalsorises 1        0        0        0          NNP        robert
                                           1          NNP          cohn
                                           2          VBD           was
                                           3           RB          once
                                           4           JJ  middleweight

In [11]:
TOKENS = TOKENS.rename(columns={'chap_num': 'chap_id'})

In [12]:
TOKENS.head()

pos      term_str
book_id         chap_num para_num sent_num token_num                   
thesunalsorises 1        0        0        0          NNP        robert
                                           1          NNP          cohn
                                           2          VBD           was
                                           3           RB          once
                                           4           JJ  middleweight

In [13]:
TOKENS.head()

pos      term_str
book_id         chap_num para_num sent_num token_num                   
thesunalsorises 1        0        0        0          NNP        robert
                                           1          NNP          cohn
                                           2          VBD           was
                                           3           RB          once
                                           4           JJ  middleweight

In [14]:
CORPUS = TOKENS

In [15]:
len(CORPUS)

90224

In [16]:
CORPUS = CORPUS[~CORPUS['term_str'].str.match('^[^a-zA-Z]+$')]
len(CORPUS)

69702

In [17]:
csv_file = f"../data/output/CORPUS.csv"
CORPUS.to_csv(csv_file)

### Extract a VOCAB table

In [18]:
VOCAB = CORPUS.term_str.value_counts().to_frame('n')
VOCAB.index.name = 'term_str'
VOCAB['max_pos'] = CORPUS.value_counts(['term_str','pos']).unstack().idxmax(1)

In [19]:
VOCAB.head()

,n,max_pos
term_str,,
the,4482,DT
i,2330,PRP
and,2285,CC
to,1544,TO
a,1532,DT


### Compute TFIFD and DFIDF

In [20]:
def create_bow(CORPUS, bag, item_type='term_str'):
    BOW = CORPUS.groupby(bag+[item_type])[item_type].count().to_frame('n')
    return BOW

In [21]:
def get_tfidf(BOW, tf_method='max', df_method='standard', item_type='term_str'):
            
    DTCM = BOW.n.unstack(fill_value=0) # Create Doc-Term Count Matrix
    
    if tf_method == 'sum':
        TF = (DTCM.T / DTCM.T.sum()).T
    elif tf_method == 'max':
        TF = (DTCM.T / DTCM.T.max()).T
    elif tf_method == 'log':
        TF = (np.log2(1 + DTCM.T)).T
    elif tf_method == 'raw':
        TF = DTCM
    elif tf_method == 'bool':
        TF = DTCM.astype('bool').astype('int')
    else:
        raise ValueError(f"TF method {tf_method} not found.")

    DF = DTCM.astype('bool').sum()
    N_docs = len(DTCM)
    
    if df_method == 'standard':
        IDF = np.log2(N_docs/DF) # This what the students were asked to use
    elif df_method == 'textbook':
        IDF = np.log2(N_docs/(DF + 1))
    elif df_method == 'sklearn':
        IDF = np.log2(N_docs/DF) + 1
    elif df_method == 'sklearn_smooth':
        IDF = np.log2((N_docs + 1)/(DF + 1)) + 1
    else:
        raise ValueError(f"DF method {df_method} not found.")
    
    TFIDF = TF * IDF
    
    DFIDF = DF * IDF

    return TFIDF, DFIDF

In [22]:
BOW = create_bow(CORPUS, ['book_id', 'chap_num'])
TFIDF, DFIDF = get_tfidf(BOW, 'max')

In [23]:
csv_file = f"../data/output/TFIDF.csv"
TFIDF.to_csv(csv_file)

In [24]:
csv_file = f"../data/output/DFIDF.csv"
DFIDF.to_csv(csv_file)

In [25]:
BOW.head()

n
book_id         chap_num term_str      
thesunalsorises 1        _fines_      1
                         a           39
                         about        6
                         absolute     1
                         accidental   1

In [26]:
csv_file = f"../data/output/BOW.csv"
BOW.to_csv(csv_file)

In [27]:
VOCAB.head(2)

,n,max_pos
term_str,,
the,4482,DT
i,2330,PRP


### Create a reduced version of the TFIDF table with only the top 1000 nouns

In [28]:
VOCAB['dfidf'] = DFIDF
VOCAB_INDEX = VOCAB[VOCAB.max_pos.isin(['NN','NNS'])].sort_values('dfidf', ascending=False).head(1000).index

In [29]:
VOCAB.head()

,n,max_pos,dfidf
term_str,,,
the,4482,DT,0.0
i,2330,PRP,0.0
and,2285,CC,0.0
to,1544,TO,0.0
a,1532,DT,0.0


In [30]:
VOCAB_INDEX

Index(['date', 'fight', 'hung', 'business', 'world', 'arcade', 'top', 'floor',
       'chaps', 'bus',
       ...
       'shells', 'odd-looking', 'miracle', 'welts', 'hardening',
       'tourist-cars', 'manufacturers', 'englishwomen', 'discovery', 'wan'],
      dtype='object', name='term_str', length=1000)

### Add labels to LIB

In [31]:
DOC = pd.DataFrame(index=TFIDF.index).join(LIB)

In [32]:
DOC.head()

genre_id author_id
book_id         chap_num                   
thesunalsorises 1         fiction    ernest
                2         fiction    ernest
                3         fiction    ernest
                4         fiction    ernest
                5         fiction    ernest

### Begin PCA Process

#### Write a function that computes PCA from a given document-term count matrix

In [33]:
def get_pca(X, k=10, norm_docs=True, center_by_mean=False, center_by_variance=False):
    # Normalize documents by their L2 norm
    if norm_docs:
        X = (X.T / norm(X, 2, axis=1)).T
    
    # Center columns by their mean
    if center_by_mean:
        X = X - X.mean()
        
    # Scale columns by their standard deviation
    if center_by_variance:
        X = X / X.std()        

    # Compute covariance matrix
    COV = X.cov()
    
    # Decompose covariance matrix into eigenvectors and eigenvalues
    eig_vals, eig_vecs = eigh(COV)
    
    # Convert eigenvectors and eigenvalues to dataframes
    EIG_VEC = pd.DataFrame(eig_vecs, index=COV.index, columns=COV.index)
    EIG_VAL = pd.DataFrame(eig_vals, index=COV.index, columns=['eig_val'])
    EIG_VAL.index.name = 'term_str'
    
    # Pick top k components
    EIG_INDEX = EIG_VAL.eig_val.sort_values(ascending=False).head(k)
    COMPS = EIG_VEC[EIG_INDEX.index].T
    COMPS_size = COMPS.shape[0]
    COMPS.index = [i for i in range(COMPS_size)]
    COMPS.index.name = 'pc_id'

    # Compute loadings
    LOADINGS = COMPS.T

    # Project documents onto components
    DCM = X.dot(LOADINGS)
    
    # Compute component information table
    COMPINF = pd.DataFrame(index=COMPS.index)

    for i in range(k):
        for j in [0, 1]:
            top_terms = ' '.join(LOADINGS.sort_values(i, ascending=bool(j)).head(10).index.to_list())
            COMPINF.loc[i, j] = top_terms
    COMPINF = COMPINF.rename(columns={0:'pos', 1:'neg'})
    
    COMPINF['eig_val'] = EIG_INDEX.reset_index(drop=True).to_frame()
    COMPINF['exp_var'] = COMPINF.eig_val / COMPINF.eig_val.sum()
    
    return LOADINGS, DCM, COMPINF

In [34]:
from scipy.linalg import eigh

In [35]:
LOADINGS, DCM, COMPINF = get_pca(TFIDF[VOCAB_INDEX], k=10, norm_docs=True, center_by_mean=False, center_by_variance=False)

In [36]:
LOADINGS.head()

pc_id,0,1,2,3,4,5,6,7,8,9
term_str,,,,,,,,,,
date,0.023580,0.008945,-0.000558,-0.005318,0.033992,0.002685,-0.006839,-0.021607,-0.005328,-0.023126
fight,-0.019143,-0.000690,0.019586,0.061700,0.052596,0.001086,0.011305,0.088121,0.008741,0.037551
hung,-0.041098,-0.012379,-0.009968,-0.006047,-0.009513,0.034095,0.003242,-0.013495,0.028858,0.010336
business,0.000219,-0.002097,-0.009068,-0.023381,-0.021129,-0.022087,-0.012388,0.024671,-0.008357,0.047760
world,0.026084,0.018248,-0.000079,0.023787,0.022288,0.060573,-0.007362,-0.010080,0.026538,-0.089602


In [37]:
DCM.head()

pc_id                            0         1         2         3         4  \
book_id         chap_num                                                     
thesunalsorises 1         0.280524  0.127038  0.453271 -0.320780 -0.048312   
                2         0.202022  0.056830  0.350477 -0.243227 -0.387445   
                3         0.145729  0.168205  0.125276  0.202494  0.337345   
                4         0.216413  0.399214 -0.555377 -0.058554 -0.018850   
                5         0.169457 -0.005661  0.100063  0.345369 -0.396702   

pc_id                            5         6         7         8         9  
book_id         chap_num                                                    
thesunalsorises 1        -0.018800 -0.208169  0.092446  0.219764 -0.028642  
                2         0.067392  0.050954  0.144870 -0.259963  0.247872  
                3         0.042590 -0.106916 -0.541812 -0.479924  0.214946  
                4        -0.112902 -0.069652 -0.076002 -0.041638  0.051871  
                5        -0.476617  0.371541 -0.302973  0.193902 -0.102212

In [38]:
COMPINF.head()

,pos,neg,eig_val,exp_var
pc_id,,,,
0,count frances magazine novel taxi mother nigge...,bull bulls cape bull-fighter dancers bull-figh...,0.095521,0.156541
1,count bull dear frances daughter champagne con...,stream road bus hills inn trout woods bag dust...,0.083749,0.137249
2,frances novel magazine mother tennis scenes li...,count concierge champagne sir stream road bed ...,0.074018,0.121301
3,service dining-car beach hotel car hors œuvres...,novel stream count magazine woods road life tr...,0.057786,0.094701
4,nigger pernod taxi bull bar corners cat river ...,life hors œuvres office questions pays novel p...,0.054907,0.089982


In [39]:
DOC.head()

genre_id author_id
book_id         chap_num                   
thesunalsorises 1         fiction    ernest
                2         fiction    ernest
                3         fiction    ernest
                4         fiction    ernest
                5         fiction    ernest

In [40]:
DOC['label'] = DOC.apply(lambda x: f"{x.name[0]}_CHAP{x.name[1]}_{x.genre_id}_{x.author_id}", 1)
DOC.head()

genre_id author_id  \
book_id         chap_num                      
thesunalsorises 1         fiction    ernest   
                2         fiction    ernest   
                3         fiction    ernest   
                4         fiction    ernest   
                5         fiction    ernest   

                                                         label  
book_id         chap_num                                        
thesunalsorises 1         thesunalsorises_CHAP1_fiction_ernest  
                2         thesunalsorises_CHAP2_fiction_ernest  
                3         thesunalsorises_CHAP3_fiction_ernest  
                4         thesunalsorises_CHAP4_fiction_ernest  
                5         thesunalsorises_CHAP5_fiction_ernest

In [41]:
csv_file = f"../data/output/LIB.csv"
DOC.to_csv(csv_file)

### Visualize PCs and Loadings

In [42]:
def vis_pcs(M, a, b, label=DOC.author_id, hover_name=DOC.label, symbol=None, size=None):
    fig = px.scatter(M, a, b, color=label, hover_name=hover_name, 
                     symbol=symbol, size=size,
                     marginal_x='box', marginal_y='box', height=800)
    fig.show()

In [43]:
def vis_loadings(a=0, b=1, hover_name='term_str'):
    X = LOADINGS
    return px.scatter(X.reset_index(), a, b, 
                      text='term_str', size=VOCAB.loc[VOCAB_INDEX].n, 
                      marginal_x='box', height=800)

In [44]:
LOADINGS.head()

pc_id,0,1,2,3,4,5,6,7,8,9
term_str,,,,,,,,,,
date,0.023580,0.008945,-0.000558,-0.005318,0.033992,0.002685,-0.006839,-0.021607,-0.005328,-0.023126
fight,-0.019143,-0.000690,0.019586,0.061700,0.052596,0.001086,0.011305,0.088121,0.008741,0.037551
hung,-0.041098,-0.012379,-0.009968,-0.006047,-0.009513,0.034095,0.003242,-0.013495,0.028858,0.010336
business,0.000219,-0.002097,-0.009068,-0.023381,-0.021129,-0.022087,-0.012388,0.024671,-0.008357,0.047760
world,0.026084,0.018248,-0.000079,0.023787,0.022288,0.060573,-0.007362,-0.010080,0.026538,-0.089602


In [45]:
VOCAB.loc[VOCAB_INDEX]

,n,max_pos,dfidf
term_str,,,
date,8,NN,10.084008
fight,18,NN,10.084008
hung,13,NN,10.084008
business,14,NN,10.084008
world,15,NN,10.084008
...,...,...,...
tourist-cars,1,NNS,4.247928
manufacturers,1,NNS,4.247928
englishwomen,1,NNS,4.247928


### PC 1 and 2

In [46]:
vis_pcs(DCM, 1, 2)

In [47]:
vis_loadings(1, 2)

### PC 2 and 3

In [48]:
vis_pcs(DCM, 2, 3)

In [49]:
vis_loadings(2, 3)

In [50]:
px.box(DCM, x=DOC.genre_id, y=0, height=800, color=DOC.genre_id)

In [51]:
vis_loadings(0, 1)

In [52]:
print(LOADINGS[1].nlargest(3))
print(LOADINGS[1].nsmallest(3))

term_str
count    0.366512
bull     0.118337
dear     0.116254
Name: 1, dtype: float64
term_str
stream   -0.313116
road     -0.299130
bus      -0.156142
Name: 1, dtype: float64


Recomputing the principle components with center_by_variance set to True.

This changes the words that appear at the extremes of the first PC, and thus changes the interpretation from above.

In [53]:
LOADINGS, DCM, COMPINF = get_pca(TFIDF[VOCAB_INDEX], k=10, norm_docs=True, center_by_mean=False, center_by_variance=True)

In [54]:
px.scatter(LOADINGS.reset_index(), 0, 1, 
                      text='term_str', size=VOCAB.loc[VOCAB_INDEX].n, 
                      marginal_x='box', height=800)

In [55]:
print(LOADINGS[1].nlargest(3))
print(LOADINGS[1].nsmallest(3))

term_str
forty      0.053580
bridge     0.050569
country    0.049910
Name: 1, dtype: float64
term_str
crowd    -0.095215
feet     -0.094210
chapel   -0.091772
Name: 1, dtype: float64


### Start Word Embedding - HW 9

In [56]:
import pandas as pd
import numpy as np
from gensim.models import word2vec
from sklearn.manifold import TSNE
import plotly_express as px

In [57]:
TOKENS = pd.read_csv(f"../data/output/nltk_tokens.csv", index_col=0)

In [58]:
TOKENS.head()

,para_num,sent_num,token_num,pos_tuple,pos,token_str,term_str
chap_num,,,,,,,
1,0,0,0,"('Robert', 'NNP')",NNP,Robert,robert
1,0,0,1,"('Cohn', 'NNP')",NNP,Cohn,cohn
1,0,0,2,"('was', 'VBD')",VBD,was,was
1,0,0,3,"('once', 'RB')",RB,once,once
1,0,0,4,"('middleweight', 'JJ')",JJ,middleweight,middleweight


In [59]:
TOKENS = TOKENS.reset_index()
book_id = 'thesunalsorises'
TOKENS['book_id'] = book_id
TOKENS = TOKENS.set_index(['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num'])

In [60]:
TOKENS.head()

pos_tuple  \
book_id         chap_num para_num sent_num token_num                           
thesunalsorises 1        0        0        0               ('Robert', 'NNP')   
                                           1                 ('Cohn', 'NNP')   
                                           2                  ('was', 'VBD')   
                                           3                  ('once', 'RB')   
                                           4          ('middleweight', 'JJ')   

                                                      pos     token_str  \
book_id         chap_num para_num sent_num token_num                      
thesunalsorises 1        0        0        0          NNP        Robert   
                                           1          NNP          Cohn   
                                           2          VBD           was   
                                           3           RB          once   
                                           4           JJ  middleweight   

                                                          term_str  
book_id         chap_num para_num sent_num token_num                
thesunalsorises 1        0        0        0                robert  
                                           1                  cohn  
                                           2                   was  
                                           3                  once  
                                           4          middleweight

In [61]:
csv_file = f"../data/output/tokens_final.csv"
TOKENS.to_csv(csv_file)

In [62]:
len(TOKENS)

90224

In [63]:
TOKENS = TOKENS[~TOKENS['term_str'].str.match('^[^a-zA-Z]+$')]
len(TOKENS)

69702

In [64]:
noun_verb_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
proper_nouns_tags = ['NNP', 'NNPS']

In [65]:
TOKENS_FILTERED = TOKENS[(TOKENS['pos'].isin(noun_verb_tags)) & (~TOKENS['pos'].isin(proper_nouns_tags))]

In [66]:
TOKENS_FILTERED.head()

pos_tuple  pos  \
book_id         chap_num para_num sent_num token_num                            
thesunalsorises 1        0        0        2              ('was', 'VBD')  VBD   
                                           5           ('boxing', 'VBG')  VBG   
                                           6          ('champion', 'NN')   NN   
                                  1        0               ('Do', 'VBP')  VBP   
                                           2             ('think', 'VB')   VB   

                                                     token_str  term_str  
book_id         chap_num para_num sent_num token_num                      
thesunalsorises 1        0        0        2               was       was  
                                           5            boxing    boxing  
                                           6          champion  champion  
                                  1        0                Do        do  
                                           2             think     think

In [67]:
OHCO =['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']

In [68]:
LIB.head()

,genre_id,author_id
book_id,,
thesunalsorises,fiction,ernest


In [69]:
LIB_og = LIB

### Generate word embeddings using GENSIM

In [70]:
# word2vec()
w2v_ernest_args = dict(
    window = 2,
    min_count = 50,
    vector_size = 256
)
# TSNE()
tsne_args = dict(
    learning_rate = 200.,
    perplexity = 20,
    n_components = 2,
    init = 'random',
    n_iter = 1000,
    random_state = 42    
)

In [71]:
class do_all_work():

    BAG = ['book_id', 'chap_num', 'para_num', 'sent_num']

    def __init__(self, tokens, w2v_args, tsne_args):
        self.TOKENS:pd.DataFrame = tokens
        self.tsne_args = tsne_args
        self.w2v_args = w2v_args
        
    def generate_model(self):
        self.extract_vocab()
        self.get_model()
        self.get_tsne_coords()
        
    def extract_vocab(self):
        self.VOCAB = self.TOKENS.term_str.value_counts().to_frame('n')
        self.VOCAB.index.name = 'term_str'
        
        self.VOCAB['pos_max'] = self.TOKENS.value_counts(['term_str','pos']).unstack().idxmax(1)
        self.VOCAB['pos_group'] = self.VOCAB.pos_max.str.slice(0,2)
        
        self.SENTS = self.TOKENS.groupby(self.BAG).term_str.apply(lambda  x: x.tolist()).reset_index()['term_str'].tolist()
        
    def get_model(self):
        self.model = word2vec.Word2Vec(self.SENTS, **self.w2v_args)

        self.VEC = pd.DataFrame(self.model.wv.get_normed_vectors(), index=self.model.wv.index_to_key)
        self.VEC.index.name = 'term_str'
        
        self.VEC = self.VEC.sort_index()

    def get_tsne_coords(self):
        self.tsne_engine = TSNE(**self.tsne_args)
        self.tsne_model = self.tsne_engine.fit_transform(self.VEC)

        self.TSNE = pd.DataFrame(self.tsne_model, columns=['x','y'], index=self.VEC.index).join(self.VOCAB, how='left')[['x','y','n','pos_group']]
        
    def plot_coords(self, n=1000):
        X = self.TSNE.reset_index()
        px.scatter(X, 'x', 'y', 
                   text='term_str', 
                   color='pos_group', 
                   hover_name='term_str',
                   size=np.log2(self.TSNE.n),
                   height=1000).update_traces(
                mode='markers+text', 
                textfont=dict(color='black', size=14, family='Arial'),
                textposition='top center').show()

    def complete_analogy(self, A, B, C, n=2):
        try:
            cols = ['term', 'sim']
            return pd.DataFrame(self.model.wv.most_similar(positive=[B, C], negative=[A])[0:n], columns=cols)
        except KeyError as e:
            print('Error:', e)
            return None

    def get_most_similar(self, positive, negative=None):
        return pd.DataFrame(self.model.wv.most_similar(positive, negative), columns=['term', 'sim'])

In [72]:
ernest = do_all_work(TOKENS_FILTERED,w2v_ernest_args,tsne_args)

In [73]:
ernest.generate_model()

In [74]:
ernest.TSNE

,x,y,n,pos_group
term_str,,,,
anything,66.769173,-219.582977,60,NN
are,284.535614,-84.208458,84,VB
asked,47.441414,-165.283890,146,VB
be,161.286621,-78.229485,231,VB
bed,-86.333221,158.462418,56,NN
...,...,...,...,...
was,22.273167,-10.245003,1155,VB
way,-57.669445,0.735964,85,NN
went,-11.791186,-4.134007,297,VB


In [75]:
ernest.plot_coords()

In [76]:
ernest.complete_analogy('drink','bulls','go', 3)

,term,sim
0,had,0.931272
1,was,0.929318
2,were,0.928737
